In [10]:
import torch.nn as nn
import torch

# Transformers
Esta es una guia que implementa los transformers, tienen una arquitectura:

![arquitectura transformer](img/arquitectura.png)

una de las partes mas importantes es la `multi-head attention`, se puede leer en profundidad en el paper:

Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Brain, G., Research, G., Jones, L., Gomez, A., Kaiser, Ł., & Polosukhin, I. (2017). Attention Is All You Need.


## en gran escala:
Los objetos nuevos y más importantes
- el bloque a la izquierda es el `Encoder`
- el bloque a la derecha es el `Decoder`

Como se observa el Encoder pasa una input y se la entrega al decoder;
otro detalle importante es que el transformer pone importancia en el `orden de las palabras`, asi que a las inputs les agregamos información posicional (embeddings o etiquetas posicionals)


## Mecanismo de Atención

### Multiheaded Attention
Tengamos la input de embeddings, 256 dimensiones por ejemplo;
la dividimos en 8 partes con 32 dimensiones cada una, cada una atravieza las `capas lineas`;
se envia por el producto punto especial, llamado `Scaled Dot-Product Attention`

Se puede leer sobre el mecanismo en el pdf incluido: `Transformers Atención.pdf`, donde explico los componentes del producto punto que actua como atención

![atencion mechanism](img/attention.png)

In [2]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        # el embed size viene a ser el largo de la oración
        # ergo afecta al tamaño de las matrices de values, keys, queries
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size debe ser divisible por las heads"


        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):

        # tamaño de la información que enviamos a la vez
        N = query.shape[0]
        # dependera del largo de la oración completa
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # pasamos los VKQ por las capas Lineles
        values = self.values(values)    # (N, value_len, embed_size)
        keys = self.keys(keys)          # (N, key_len, embed_size)
        queries = self.queries(query)   # (N, query_len, embed_size)

        # dividimos los embeddings en los heads
        # agregando otra dimensión, con tamaño head_dim
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)

        # Multiplicación de matrices ------
        producto_punto = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape:    (N, query_len, heads, heads_dim)
        # keys shape:       (N, key_len, heads, heads_dim)
        # producto_punto:   (N, heads, query_len, key_len)

        # Si el elemento del mask es 0, entonces 
        if mask is not None:    # cambiamos el elemento por -infinito
            producto_punto = producto_punto.masked_fill(mask == 0, float("-1e20"))

        # dividimos por la raiz del embed size
        # y aplicamos softmax
        # dim=3, se encargara de ver en que dimensión de producto_punto normalizar
        # en este caso por la `key_len`
        softmax_out = torch.softmax(producto_punto / (self.embed_size**(1 / 2)), 
            dim=3)
        # attention shape: (N, heads, query_len, key_len)

        # Multiplicación de matrices ------
        # softmax * V
        # y haceos el paso de concatenación
        attention = torch.einsum("nhql,nlhd->nqhd", [softmax_out, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # attention: (N, query_len, heads, head_dim), then

        # la pasamos por la fully connected network
        attention = self.fc_out(attention)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

        return attention

### Bloque de Transformer, 
![encoder](img/transformer_block_encoder.png)

In [3]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()

        # usamos la atención que implementamos
        self.attention = SelfAttention(embed_size, heads)
        # la pasamos por la primera normalización
        self.norm1 = nn.LayerNorm(embed_size)
        # el feed forward, que es una FFN secuencial simple
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        # segunda normalizacion
        self.norm2 = nn.LayerNorm(embed_size)
        # iterativamente quitamos elementos para evitar co dependencia
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        # el output de attention
        attention = self.attention(value, key, query, mask)

        # attention + query es la conexión extra que llega al addnorm
        # de manera de mantener algo de información de la query
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


### Encoder
Combinemos ahora el transformer block para formular un `encoder`

![encoder](img/encoder.png)



In [4]:
class Encoder(nn.Module):

    # los hiperparametros del modelo
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):
        
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        # los embedding requiere el tamaño del vocabulario maximo
        # asi como el largo maximo de la oración
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        # N x
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        # el dropout para evitar codependencia y overfitting
        self.dropout = nn.Dropout(dropout)


    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


### Decoder Block

![decoder](img/decoder.png)

In [6]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        # masked multiheaded attention
        attention = self.attention(x, x, x, trg_mask)
        # nos entrega un query  que se junta con la input original
        query = self.dropout(self.norm(attention + x))
        # luego pasa por el transformers block
        # el cual parte de la multiheaded attention hacia arriba
        out = self.transformer_block(value, key, query, src_mask)
        return out

### Decoder
Donde incluimos los $N \times$ `DecoderBlocks`

In [13]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        # muy similar a al encoder el comienzo
        # solo que vamos por el target
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        # Tenemos entonces el N x DecoderBlocks 
        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out


## Transformer
Combinando ahora todas las piezas, tenemos el transformer

In [14]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0.1,
        device="cpu",
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out





In [15]:
# poniendolo a prueba
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# las oraciones de input
oracion_input = torch.tensor([
    [1,5,6,4,3,9,5,2,0],
    [1,8,7,3,4,5,6,7,2]
]).to(device)

target = torch.tensor([
    [1,7,4,3,5,9,2,0],
    [1,5,6,2,4,7,6,2]
]).to(device)

src_pad_idx = 0 # que representa un espacio en blanco, osea padding
trg_pad_idx = 0

src_vocab_size = 10 # tamaño del vocabulario
trg_vocab_size = 10

model = Transformer(src_vocab_size, trg_vocab_size, 
    src_pad_idx, trg_pad_idx).to(device)

out = model(oracion_input, target)
print(out.shape)

torch.Size([2, 8, 10])
